In [1]:
# Necessary for importing modules from a sub-directory
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
if os.getcwd().split('/')[-1] == 'utils':
    print("CHANGE DIR TO ROOT")
    os.chdir(r"../")

import pickle
import pandas
import nltk
import re
from time import time

from preprocess import article_text_to_tfidf
from preprocess import unnest_inner_word_list, tokenize_df, udpipe_tokenize_df
from spacy.lang.sv import Swedish
from gensim.models.phrases import Phrases, Phraser

CHANGE DIR TO ROOT
Already downloaded a model for the 'sv' language


In [2]:
# INPUT:
# Load dataset (This should be a large randomized dataset)
DATASET_TYPE = 'random'
DATASET_SIZE = '4000'

corpus_1 = pickle.load(open(f'var/efselab_parsed_{DATASET_TYPE}_{DATASET_SIZE}_1.pkl', 'rb'))
corpus_2 = pickle.load(open(f'var/efselab_parsed_{DATASET_TYPE}_{DATASET_SIZE}_2.pkl', 'rb'))
corpus_3 = pickle.load(open(f'var/efselab_parsed_{DATASET_TYPE}_{DATASET_SIZE}_3.pkl', 'rb'))
corpus_4 = pickle.load(open(f'var/efselab_parsed_{DATASET_TYPE}_{DATASET_SIZE}_4.pkl', 'rb'))
corpus_5 = pickle.load(open(f'var/efselab_parsed_{DATASET_TYPE}_{DATASET_SIZE}_5.pkl', 'rb'))
corpus_ref_ext = pickle.load(open('var/efselab_parsed_ref.pkl', 'rb'))
#corpus_3 = pickle.load(open(f'var/efselab_parsed_{DATASET_TYPE}_{DATASET_SIZE}_3.pkl', 'rb'))

#load 4000random-i i=1:6, merge these sets

In [3]:
#reference_corpus = corpus_1 + corpus_2 + corpus_3 + corpus_4 + corpus_5
reference_corpus = corpus_ref_ext
print(len(reference_corpus))

20000


In [ ]:
# Define NGRAM, if applicable
def get_phraser_model(corpus, level=3):
    if len(corpus) < 1000:
        bi_mc, bi_t = 10, 70
        tri_mc, tri_t = 15, 50
    elif (len(corpus) < 4000):
        bi_mc, bi_t = 25, 250
        tri_mc, tri_t = 25, 200
    else:
        bi_mc, bi_t = 35, 800
        tri_mc, tri_t = 15, 50
        
    ngram = None
    if level >= 2:
        phrases = Phrases(corpus, min_count=bi_mc, threshold=bi_t, delimiter=b'_')
        phraser = Phraser(phrases)
    if level >= 3:
        bigramed_corpus = [phraser[doc] for doc in corpus]
        phrases = Phrases(bigramed_corpus, min_count=tri_mc, threshold=tri_t, delimiter=b'_')
        phraser = Phraser(phrases)
    return phraser

phraser = get_phraser_model(reference_corpus, level=3)


In [ ]:
#single_phrased_reference_corpus = [phraser[doc] for doc in reference_corpus]
double_phrased_reference_corpus = [phraser[phraser[doc]] for doc in reference_corpus]

In [38]:
default_uniques = set()
for doc in reference_corpus:
    for word in doc:
        if word.count('_') >= 1:
            default_uniques.add(word)


In [40]:
uniques = set()
for doc in double_phrased_reference_corpus:
    for word in doc:
        if word.count('_') == 1 and word not in default_uniques:
            uniques.add(word)
print('#found grams', len(uniques))
print()
for w in uniques:
    print(w)

#found grams 2961

aktie_stiga
gui_minhai
orkan_irma
michel_temer
stämma_överens
orionteatern_katarina
allsång_skansen
erika_lust
sätta_ugn
t_eriksplan
per_hagman
svartmangatan_t
rolling_stone
torg_t
richard_thaler
yvonne_lombard
mikael_lustig
north_carolina
magnus_betnér
lotta_edholm
konservativ_tory
var_femte
hög_grad
debaser_strand
dn_gratulera
väster_götaland
vit_duk
à_la
bb_stockholm
lars_lerin
stor_nygatan
mårten_castenfors
mikael_sjöberg
normalt_sedd
meter_räls
gruppledare_riksdag
teknisk_undersökning
inbördes_möte
rådhus_kl
dålig_skick
inget_tyda
binda_samman
lyfta_fram
lars_heikensten
göran_hägglund
lars_knutsson
doug_jones
birgitta_forsberg
taylor_swift
kort_varsel
anna_brema
sara_landström
kristoffer_nordfeldt
logga_in
orkan_harvey
caroline_cederquist
hårt_drabbad
hanna_wigh
grand_hôtel
south_carolina
don_giovanni
he_was
nbc_news
åke_cato
immanuelskyrkan_kungstensgatan
åklagare_yrka
teodora_del
melvin_samsom
hård_kritik
israel_premiärminister
erika_kinsey
david_cameron
obepr

säkerhetsrådgivare_michael
professor_psykologi
ivar_arpi
anna_rantala
grand_prix
t_östermalmstorg
citizen_schein
skada_allvarligt
mark_dougherty
ond_cirkel
bokstavligt_talad
olsson_olsén
bertil_gustafsson
mycken_peng
marit_björgen
pia_sundhage
champion_league-final
kim_jong-nam
stående_ovation
bodil_malmsten
tim_kaine
stockholm_universitet
professor_statsvetenskap
isaac_kiese
motsvarande_siffra
teknisk_utveckling
hamngatan_t
braden_holtby
t_göran
lotta_schelin
gemensam_alliansbudget
sara_danius
peer_gynt
ingela_lind
föra_ambulans
internationell_kvinnodag
hjalmar_söderberg
rikard_norling
jesper_nelin
göta_hovrätt
stockholm_stadsbibliotek
komfort_utrymme
klubb_webbplats
blank_nej
anders_behring
skaka_huvud
greta_garbo
harvey_weinstein
martin_gelin
riksenhet_säkerhetsmål
jerker_virdborg
von_koskull
veronica_maggio
ständigt_närvarande
bred_kavanaugh
wall_street
anders_ygema
johan_cronema
nikkei_index
dra_slutsats
foto_ap
lägga_beslag
grov_stöld
lag_hasselborg
marija_banusic
hedvig_lindahl


In [15]:
## COULD ALSO LOAD .txt file
ngrams = []
path = os.getcwd() + '/ngrams_gold_saved/'
for filename in os.listdir(path):
    with open(os.path.join(path, filename), 'r') as f: # open in readonly mode
        lines = f.readlines()
        for line in lines:
            ngrams.append(line.strip())
print(len(ngrams))
ngrams_unique = sorted(list(set(ngrams)))
print(len(ngrams_unique))


f=open('ngrams_gold_saved/gold_ngrams.txt','w')
for ngram in ngrams_unique:
    f.write(ngram+'\n')



971
443


In [131]:
# GOLD STANDARD NGRAMS
# INPUT: READ TXT FILE OR SIMILAR THAT HAS ONE ROW PER GOLD STANDARD NGRAM
print('LEN:', len(gold_standard_grams))
gold_ngrams = {
    'bigrams': [],
    'trigrams': [],
}

for ngram in uniques:
    ngram_split = ngram.split('_')
    if len(ngram_split) == 2:
        gold_ngrams['bigrams'].append(tuple(ngram_split))
    if len(ngram_split) == 3:
        gold_ngrams['trigrams'].append(tuple((ngram_split)))

print(gold_ngrams['bigrams'][:3] + [('...')],)
print(gold_ngrams['trigrams'][:3]+ [('...')],)

# YIELDS: DICT OF BIGRAMS AND TRIGRAMS AS TWO LISTS OF TUPLES OF TERMS
# OUTPUT: DICT WITH 2 keys:
# {
#  'bigrams': <list of tuples of 2 terms, each tuple represents a bigram>
#  'trigrams': <list of tuples of 3 terms, each tuple represents a trigram>
# }

LEN: 2
[('robin', 'olsen'), ('östersund', 'fk'), ('johan', 'cronema'), '...']
[('new', 'york', 'times'), ('albert', 'bonnier', 'förlag'), ('tour', 'de', 'ski'), '...']


In [132]:
gold_ngrams

{'bigrams': [('robin', 'olsen'),
  ('östersund', 'fk'),
  ('johan', 'cronema'),
  ('malmö', 'ff'),
  ('charlotte', 'kalla'),
  ('roger', 'federer'),
  ('presstalesperson', 'stockholmspolis'),
  ('afc', 'eskilstuna'),
  ('europa', 'league'),
  ('grand', 'prix'),
  ('therese', 'johaug'),
  ('nilla', 'fischer'),
  ('janne', 'andersson'),
  ('rikard', 'norling'),
  ('hanna', 'öberg'),
  ('albert', 'bonnier'),
  ('ebba', 'andersson'),
  ('adam', 'reideborn'),
  ('ewert', 'gren'),
  ('huvud', 'tagen'),
  ('elias', 'pettersson'),
  ('olympisk', 'kommitté'),
  ('ifk', 'göteborg'),
  ('donald', 'trump'),
  ('baryard', 'johnsson'),
  ('st', 'louis'),
  ('andreas', 'granqvist'),
  ('susanne', 'ljung'),
  ('sun', 'yang'),
  ('eric', 'torell'),
  ('maja', 'dahlqvist'),
  ('robert', 'ohlsson'),
  ('cissi', 'wallin'),
  ('alexander', 'isak'),
  ('west', 'ham'),
  ('rebecca', 'peterson'),
  ('calle', 'halfvarsson'),
  ('jean-claude', 'arnault'),
  ('nilsson', 'lindelöf'),
  ('kosovare', 'asllani'),
  

In [133]:
# SAVE TO PICKLE
root = os.getcwd()
GOLD_NGRAM_PICKLE_OUTPUT = f'{root}/pickles/gold_ngrams.pkl'
pickle.dump(gold_ngrams, open(GOLD_NGRAM_PICKLE_OUTPUT, 'wb'))

In [ ]:
##############################################################################################################

In [124]:
# LOAD EXTERNAL NGRAMS (FROM NER TAGS FOR EXAMPLE)
# Logic for merging these, for flattening and building Phraser
custom_bigrams = [
    ('donald', 'trump'),
    ('joe', 'biden'),
    ('mänsklig', 'rättighet'),
    ('artificiell', 'intelligens'),
    ('washington', 'post'),
]

custom_trigrams = [
    ('black', 'lives', 'matter'),
    ('wall', 'street', 'journal'),
]

custom_ngrams = {
    'bigrams': custom_bigrams,
    'trigrams': custom_trigrams,   
}

In [126]:
# COMBINE GRAMS
merged_bigrams = gold_ngrams['bigrams'] + custom_ngrams['bigrams']
merged_trigrams = gold_ngrams['trigrams'] + custom_ngrams['trigrams']
print(merged_bigrams[:2], '...', merged_bigrams[-2:])
print(merged_trigrams[:2], '...', merged_trigrams[-2:])

distinct_bigrams = [list(gram) for gram in set(merged_bigrams)]
distinct_trigrams = [list(gram) for gram in set(merged_trigrams)]


[('robin', 'olsen'), ('östersund', 'fk')] ... [('artificiell', 'intelligens'), ('washington', 'post')]
[('new', 'york', 'times'), ('albert', 'bonnier', 'förlag')] ... [('black', 'lives', 'matter'), ('wall', 'street', 'journal')]


In [127]:
#flattened_gold_ngrams = gold_ngrams['bigrams'] + \
#            [gram[0:2] for gram in gold_ngrams['trigrams'] if gram[0:2] not in gold_ngrams['bigrams']] + \
#            [['_'.join(gram[0:2]), gram[2]] for gram in gold_ngrams['trigrams'] ]

flat_merged_ngrams = distinct_bigrams + \
            [gram[0:2] for gram in distinct_trigrams if gram[0:2] not in distinct_bigrams] + \
            [['_'.join(gram[0:2]), gram[2]] for gram in distinct_trigrams] + \
            [[gram[0], '_'.join(gram[1:3])] for gram in distinct_trigrams]

In [128]:
flat_merged_ngrams[-20:]

[['afc', 'eskilstuna'],
 ['sarah', 'sjöström'],
 ['joe', 'biden'],
 ['eric', 'torell'],
 ['black', 'lives'],
 ['victor', 'nilsson'],
 ['tour', 'de'],
 ['wall', 'street'],
 ['black_lives', 'matter'],
 ['victor_nilsson', 'lindelöf'],
 ['new_york', 'times'],
 ['tour_de', 'ski'],
 ['albert_bonnier', 'förlag'],
 ['wall_street', 'journal'],
 ['black', 'lives_matter'],
 ['victor', 'nilsson_lindelöf'],
 ['new', 'york_times'],
 ['tour', 'de_ski'],
 ['albert', 'bonnier_förlag'],
 ['wall', 'street_journal']]

In [129]:
gold_Phrases = Phrases(flat_merged_ngrams, min_count=0.00000001, threshold=0.0000001, delimiter=b'_')


In [96]:
for i, (phrase, score) in enumerate(gold_Phrases.export_phrases(flat_merged_ngrams)):
    print(i, phrase, score)

0 b'robin olsen' 205.99999794
1 b'\xc3\xb6stersund fk' 411.99999588
2 b'johan cronema' 205.99999794
3 b'malm\xc3\xb6 ff' 411.99999588
4 b'charlotte kalla' 411.99999588
5 b'roger federer' 411.99999588
6 b'presstalesperson stockholmspolis' 411.99999588
7 b'afc eskilstuna' 411.99999588
8 b'europa league' 102.99999897
9 b'grand prix' 411.99999588
10 b'therese johaug' 411.99999588
11 b'nilla fischer' 411.99999588
12 b'janne andersson' 205.99999794
13 b'rikard norling' 411.99999588
14 b'hanna \xc3\xb6berg' 411.99999588
15 b'albert bonnier' 823.99999588
16 b'ebba andersson' 205.99999794
17 b'adam reideborn' 411.99999588
18 b'ewert gren' 411.99999588
19 b'huvud tagen' 411.99999588
20 b'elias pettersson' 411.99999588
21 b'olympisk kommitt\xc3\xa9' 411.99999588
22 b'ifk g\xc3\xb6teborg' 205.99999794
23 b'donald trump' 205.99999897
24 b'baryard johnsson' 411.99999588
25 b'st louis' 411.99999588
26 b'andreas granqvist' 411.99999588
27 b'susanne ljung' 411.99999588
28 b'sun yang' 411.99999588
29 b'

In [98]:
gold_Phrases[['cissi', 'wallin','new', 'york','new_york','times', 'donald', 'trump', 'black_lives', 'matter']]

['cissi_wallin',
 'new_york',
 'new_york_times',
 'donald_trump',
 'black_lives_matter']

In [1]:
custom_bigrams = [
    ('donald', 'trump'),
    ('joe', 'biden'),
    ('mänsklig', 'rättighet'),
    ('artificiell', 'intelligens'),
    ('washington', 'post'),
]

In [2]:
custom_trigrams = [
    ('black', 'lives'),
    ('black_lives', 'matter'),
    ('wall', 'street'),
    ('wall_street', 'journal'),
]